In [1]:
import numpy as np
import pandas as pd
from doubleml.data.panel_data import DoubleMLPanelData
from doubleml.plm.plpr import DoubleMLPLPR
from sklearn.linear_model import LassoCV
from sklearn.base import clone
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
# from doubleml.plm.utils._plpr_util import extend_data, cre_fct, fd_fct, wd_fct
from doubleml.plm.datasets.dgp_plpr_CP2025 import make_plpr_CP2025
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
import warnings
warnings.filterwarnings("ignore")

In [2]:
class PolyPlus(BaseEstimator, TransformerMixin):
    """PolynomialFeatures(degree=k) and additional terms x_i^(k+1)."""

    def __init__(self, degree=2, interaction_only=False, include_bias=False):
        self.degree = degree
        self.extra_degree = degree + 1
        self.interaction_only = interaction_only
        self.include_bias = include_bias
        self.poly = PolynomialFeatures(degree=degree, interaction_only=interaction_only, include_bias=include_bias)

    def fit(self, X, y=None):
        self.poly.fit(X)
        self.n_features_in_ = X.shape[1]
        return self

    def transform(self, X):
        X = np.asarray(X)
        X_poly = self.poly.transform(X)
        X_extra = X ** self.extra_degree
        return np.hstack([X_poly, X_extra])

    def get_feature_names_out(self, input_features=None):
        input_features = np.array(
            input_features
            if input_features is not None
            else [f"x{i}" for i in range(self.n_features_in_)]
        )
        poly_names = self.poly.get_feature_names_out(input_features)
        extra_names = [f"{name}^{self.extra_degree}" for name in input_features]
        return np.concatenate([poly_names, extra_names])

In [3]:
dim_x = 30
indices_x = [x for x in range(dim_x)]
indices_x_tr = [x + dim_x for x in indices_x]

preprocessor = ColumnTransformer([
    ('poly_x', make_pipeline(
        PolyPlus(degree=2, include_bias=False, interaction_only=False)
    ), indices_x),       
    ('poly_x_mean', make_pipeline(
        PolyPlus(degree=2, include_bias=False, interaction_only=False)
    ), indices_x_tr) 
], remainder='passthrough')

preprocessor_wg = ColumnTransformer([
    ('poly_x', make_pipeline(
        PolyPlus(degree=2, include_bias=False, interaction_only=False)
    ), indices_x),       
], remainder='passthrough')

ml_lasso = make_pipeline(
    preprocessor,
    StandardScaler(),
    LassoCV(n_alphas=20, cv=2, n_jobs=5)
)

ml_lasso_wg = make_pipeline(
    preprocessor_wg,
    StandardScaler(),
    LassoCV(n_alphas=20, cv=2, n_jobs=5)
)

ml_cart = DecisionTreeRegressor()

ml_boost = LGBMRegressor(verbose=-1, 
                         n_estimators=100, 
                         learning_rate=0.3,
                         min_child_samples=1) 

In [4]:
data = make_plpr_CP2025(num_id=100, dgp_type='dgp1')
data

,id,time,y,d,x1,x2,x3,x4,x5,x6,...,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30
0,1,1,-5.891762,-4.574437,-1.203377,-0.087056,-3.875989,-2.060015,1.924513,1.564501,...,-1.697513,-2.509320,-0.727138,-2.393134,0.334781,2.097534,1.942009,1.649557,-0.612257,-4.331109
1,1,2,0.601641,1.217127,-1.076318,2.226439,0.379887,-2.491481,-1.446766,-0.000182,...,-0.185932,-0.491894,1.320808,-2.888978,0.296153,-0.209147,-1.066396,-2.232003,3.217619,-0.660709
2,1,3,-5.336432,-4.084917,1.684777,-2.117207,-4.038299,1.196702,4.320428,1.543303,...,-1.896694,2.950372,2.266257,-1.962670,1.913956,-3.847482,0.914604,-1.721561,-0.954810,0.407410
3,1,4,-0.478058,0.043192,0.978425,-2.568042,-1.001187,-2.151350,0.973693,-1.286461,...,-1.148923,-3.388272,1.121507,4.753065,1.424797,-2.345737,-0.693004,-1.618859,1.668621,4.571664
4,1,5,-1.223138,-1.544670,1.398526,3.567711,-1.353898,-2.226735,3.713345,0.675101,...,-1.837662,2.941762,2.061895,0.853080,-0.244278,1.263040,-2.011630,-0.826488,-0.887181,-1.935609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,100,6,-2.734367,-0.265121,-3.045957,1.798310,-1.485007,-2.107084,2.983506,1.469852,...,4.836053,1.932549,2.307781,-2.536377,1.150598,1.052909,-0.969657,1.266473,-3.177021,-3.070155
996,100,7,5.307034,3.585023,-2.274700,-2.499113,2.116755,0.478902,-0.248561,-0.957826,...,-2.799372,1.598447,1.972620,-1.888645,1.237270,3.644984,0.054862,0.615274,-0.432120,-0.046949
997,100,8,-4.476127,-2.751544,-0.716859,-1.263491,-2.826469,-0.954049,-1.237438,-1.200074,...,1.527836,-1.918927,-0.381272,2.065848,0.723859,-0.711546,0.930980,0.883152,-0.324217,0.053768
998,100,9,5.191475,4.985730,-2.840246,0.931855,3.070040,2.700103,1.214848,2.846577,...,-1.662106,0.583185,2.117253,-0.429837,-1.983224,-1.249148,5.170035,3.022710,3.091618,-2.210554


In [ ]:
def run_sim(n_reps, num_id, dim_x=30, theta=0.5, dgp_type='dgp3'):

    approaches = ["cre_general", "cre_normal", "fd_exact", "wg_approx"]
    models  = ["lasso", "cart", "boost"]

    res = {
        d: {
            m: np.full((n_reps, 3), np.nan)
            for m in models
        }
        for d in approaches
    }

    x_cols = [f"x{i+1}" for i in range(dim_x)]

    def run_single_dml(dml_data, ml, approach, grid=None):
        est = DoubleMLPLPR(dml_data, clone(ml), clone(ml), approach=approach, n_folds=5)

        if grid is not None:
            est.tune(param_grids=grid, search_mode='randomized_search',
                     n_iter_randomized_search=5, n_jobs_cv=5)

        est.fit()

        coef_err = est.coef[0] - theta
        se       = est.se[0]
        conf     = est.confint()
        covered  = (conf['2.5 %'].iloc[0] <= theta) & (conf['97.5 %'].iloc[0] >= theta)

        return coef_err, se, covered

    for i in range(n_reps):

        print(f"\rProcessing: {round((i+1)/n_reps*100, 3)} %", end="")

        cart_grid = {'ml_l': {'ccp_alpha': np.random.choice(np.arange(0.002, 0.052 , 0.002), 5, replace=False),
                              'max_depth': np.random.choice(np.arange(2, 11 , 1), 5, replace=False)},
                     'ml_m': {'ccp_alpha': np.random.choice(np.arange(0.002, 0.052 , 0.002), 5, replace=False),
                              'max_depth': np.random.choice(np.arange(2, 11 , 1), 5, replace=False)}}

        boost_grid= {'ml_l': {'reg_lambda': np.random.choice(np.arange(0.2, 2 , 0.2), 5, replace=False), 
                              'max_depth': np.random.choice(np.arange(2, 11 , 1), 5, replace=False)},
                     'ml_m': {'reg_lambda': np.random.choice(np.arange(0.2, 2 , 0.2), 5, replace=False),
                              'max_depth': np.random.choice(np.arange(2, 11 , 1), 5, replace=False)}}

        data = make_plpr_CP2025(num_id=num_id, dim_x=dim_x, theta=theta, dgp_type=dgp_type)
        dml_data = DoubleMLPanelData(data, y_col='y', d_cols='d', t_col='time', id_col='id', x_cols=x_cols, static_panel=True)

        # CRE general
        res['cre_general']['lasso'][i, :] = run_single_dml(dml_data, ml_lasso, 'cre_general', grid=None)
        res['cre_general']['cart'][i, :] = run_single_dml(dml_data, ml_cart, 'cre_general', grid=cart_grid)
        res['cre_general']['boost'][i, :] = run_single_dml(dml_data, ml_boost, 'cre_general', grid=boost_grid)

        # CRE normal
        res['cre_normal']['lasso'][i, :] = run_single_dml(dml_data, ml_lasso, 'cre_normal', grid=None)
        res['cre_normal']['cart'][i, :] = run_single_dml(dml_data, ml_cart, 'cre_normal', grid=cart_grid)
        res['cre_normal']['boost'][i, :] = run_single_dml(dml_data, ml_boost, 'cre_normal', grid=boost_grid)

        # FD
        res['fd_exact']['lasso'][i, :] = run_single_dml(dml_data, ml_lasso, 'fd_exact', grid=None)
        res['fd_exact']['cart'][i, :] = run_single_dml(dml_data, ml_cart, 'fd_exact', grid=cart_grid)
        res['fd_exact']['boost'][i, :] = run_single_dml(dml_data, ml_boost, 'fd_exact', grid=boost_grid)

        # WD
        res['wg_approx']['lasso'][i, :] = run_single_dml(dml_data, ml_lasso_wg, 'wg_approx', grid=None)
        res['wg_approx']['cart'][i, :] = run_single_dml(dml_data, ml_cart, 'wg_approx', grid=cart_grid)
        res['wg_approx']['boost'][i, :] = run_single_dml(dml_data, ml_boost, 'wg_approx', grid=boost_grid)

    # summary
    rows = []
    index = []

    for approach, models_dict in res.items():
        for model, arr in models_dict.items():

            bias = np.mean(arr[:, 0])
            se_mean = np.mean(arr[:, 1])
            sd = np.std(arr[:, 1])
            coverage = np.mean(arr[:, 2])
            se_over_sd = sd / se_mean if se_mean > 0 else np.nan
            rmse = np.sqrt(np.mean(arr[:, 0]**2))

            rows.append([bias, rmse, se_over_sd, coverage])
            index.append((approach, model))      

    summary = pd.DataFrame(
        rows,
        index=pd.MultiIndex.from_tuples(index, names=["Approach", "ML Model"]),
        columns=["Bias", "RMSE", "SE/SD", "Coverage"]
    )

    return summary

In [ ]:
np.random.seed(123)

res_dgp1 = run_sim(n_reps=100, num_id=100, theta=0.5, dgp_type='dgp1')
res_dgp1

Processing: 100.0 %

Bias      RMSE     SE/SD  Coverage
Approach    ML Model                                        
cre_general lasso     0.024030  0.040756  0.076669      0.92
            cart     -0.033091  0.069709  0.112711      0.75
            boost    -0.041728  0.062508  0.114469      0.76
cre_normal  lasso     0.094727  0.103281  0.111451      0.41
            cart     -0.015669  0.072878  0.125903      0.89
            boost     0.051158  0.081405  0.128427      0.90
fd_exact    lasso     0.021691  0.044563  0.084716      0.90
            cart      0.076125  0.105962  0.099474      0.62
            boost     0.010003  0.049025  0.116624      0.87
wg_approx   lasso     0.003884  0.032382  0.077256      0.97
            cart     -0.002517  0.048548  0.090029      0.87
            boost    -0.042819  0.058042  0.086614      0.71

In [ ]:
np.random.seed(123)

res_dgp2 = run_sim(n_reps=100, num_id=100, theta=0.5, dgp_type='dgp2')
res_dgp2

Processing: 100.0 %

Bias      RMSE     SE/SD  Coverage
Approach    ML Model                                        
cre_general lasso    -0.005823  0.027244  0.073960      0.95
            cart     -0.109613  0.122162  0.106904      0.24
            boost    -0.063039  0.075676  0.081278      0.50
cre_normal  lasso     0.070362  0.076794  0.087415      0.40
            cart     -0.028230  0.073113  0.182335      0.94
            boost    -0.016764  0.060976  0.183879      0.89
fd_exact    lasso    -0.002715  0.032797  0.091710      0.95
            cart     -0.074209  0.083379  0.099775      0.47
            boost    -0.062527  0.076963  0.085480      0.58
wg_approx   lasso    -0.003015  0.027330  0.074295      0.95
            cart     -0.024405  0.038327  0.079555      0.90
            boost    -0.056258  0.067245  0.083798      0.52

In [ ]:
np.random.seed(123)

run_sim(n_reps=100, num_id=100, theta=0.5, dgp_type='dgp3')

Processing: 100.0 %

Bias      RMSE     SE/SD  Coverage
Approach    ML Model                                        
cre_general lasso     0.016582  0.037272  0.080832      0.94
            cart      0.493280  0.534554  0.275800      0.05
            boost     0.443519  0.451916  0.180075      0.00
cre_normal  lasso     0.083721  0.100538  0.129024      0.71
            cart      0.413418  0.468509  0.218055      0.13
            boost     0.385419  0.393097  0.185301      0.00
fd_exact    lasso     0.014759  0.041217  0.086594      0.92
            cart      0.658754  0.673590  0.259508      0.00
            boost     0.588552  0.593496  0.134829      0.00
wg_approx   lasso     0.630003  0.630942  0.099154      0.00
            cart      0.515894  0.532497  0.227144      0.01
            boost     0.593587  0.597220  0.137237      0.00